# Taxi ride duration prediction
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [1]:
import urllib

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

* 1054112
* 1154112
* 1254112
* 1354112

In [2]:
# Set needed data URLs and filenames
jan_data_fname, jan_data_url = (
    "../local/data/jan_data.parquet",
    "https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet"
)
feb_data_fname, feb_data_url = (
    "../local/data/feb_data.parquet",
    "https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet"
)

In [3]:
%%capture
# Download data
urllib.request.urlretrieve(jan_data_url, jan_data_fname)
urllib.request.urlretrieve(feb_data_url, feb_data_fname)

In [4]:
# Read data from January
jan_df = pd.read_parquet(jan_data_fname)
n_records_original = len(jan_df.index)

In [5]:
# Answer
print(f"How many records are there? {n_records_original}")

How many records are there? 1154112


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the average trip duration in January?

* 15.16
* 19.16
* 24.16
* 29.16

In [6]:
# Convert pick-up/drop-off times to datetime format and build new duration column
jan_df.pickup_datetime = pd.to_datetime(jan_df.pickup_datetime)
jan_df.dropOff_datetime = pd.to_datetime(jan_df.dropOff_datetime)
jan_df["duration"] = jan_df.dropOff_datetime - jan_df.pickup_datetime

In [7]:
# Convert duration from datetime to minutes
jan_df.duration = jan_df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
# Answer
print(f"What's the average trip duration in January? {jan_df.duration.mean()}")

What's the average trip duration in January? 19.167224093791006


## Data preparation

Check the distribution of the duration variable. There are some outliers. 

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop? 

In [9]:
# Filter outliers
jan_df = jan_df[(jan_df.duration >= 1) & (jan_df.duration <= 60)]
n_records_filtered = len(jan_df.index)

In [10]:
# Answer
print(f"How many records did you drop? {n_records_original - n_records_filtered}")

How many records did you drop? 44286


## Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs. 

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

* 53%
* 63%
* 73%
* 83%

In [11]:
%%capture
# Replace NaNs with -1
jan_df.PUlocationID = jan_df.PUlocationID.fillna(-1)
jan_df.DOlocationID = jan_df.DOlocationID.fillna(-1)

In [12]:
# Count number of missing pick-up location values
n_missing = (jan_df.PUlocationID == -1).sum()

In [13]:
# Answer
print(f"How many records did you drop? {(n_missing / n_records_filtered) * 100}%")

How many records did you drop? 83.52732770722618%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

* 2
* 152
* 352
* 525
* 725

In [14]:
# Convert categorical values to strings
categorical = ["PUlocationID", "DOlocationID"]
jan_df[categorical] = jan_df[categorical].astype(str)

In [15]:
# Turn dataframe into list of dicts
train_dict = jan_df[categorical].to_dict(orient="records")

In [16]:
# Fit dictionary vectorizer
dict_vec = DictVectorizer()
x_train = dict_vec.fit_transform(train_dict)

In [17]:
# Answer
print(f"What's the dimensionality of this matrix? {x_train.shape[1]}")

What's the dimensionality of this matrix? 525


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 5.52
* 10.52
* 15.52
* 20.52

In [18]:
# Extract ground truth data
y_train = jan_df.duration.values

In [19]:
%%capture
# Train a linear regressor
lr = LinearRegression()
lr.fit(x_train, y_train)

In [20]:
# Measure RMSE on training data
y_pred = lr.predict(x_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)

In [21]:
# Answer
print(f"What's the RMSE on train? {rmse}")

What's the RMSE on train? 10.528519388409808


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021). 

What's the RMSE on validation?

* 6.01
* 11.01
* 16.01
* 21.01

In [22]:
# Get validation data ready
feb_df = pd.read_parquet(feb_data_fname)

feb_df.pickup_datetime = pd.to_datetime(feb_df.pickup_datetime)
feb_df.dropOff_datetime = pd.to_datetime(feb_df.dropOff_datetime)
feb_df["duration"] = feb_df.dropOff_datetime - feb_df.pickup_datetime
feb_df.duration = feb_df.duration.apply(lambda td: td.total_seconds() / 60)

feb_df = feb_df[(feb_df.duration >= 1) & (feb_df.duration <= 60)]

feb_df.PUlocationID = feb_df.PUlocationID.fillna(-1)
feb_df.DOlocationID = feb_df.DOlocationID.fillna(-1)

feb_df[categorical] = feb_df[categorical].astype(str)

val_dict = feb_df[categorical].to_dict(orient="records")

x_val = dict_vec.transform(val_dict)

y_val = feb_df.duration.values

In [23]:
# Measure RMSE on validation data
y_pred = lr.predict(x_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)

AttributeError: 'DictVectorizer' object has no attribute 'ndim'

In [ ]:
# Answer
print(f"What's the RMSE on validation? {rmse}")